### Agenda:
1. Data Loading & Preprocessing
   - Missing values handling
   - Date features creation
   - Train/Test split
   - Scaling
   - Sequences
   - Data Loader (incl. indexing for Basisformer)
2. Experimental Design
    - Benchmark Models
      - Linear Regression
      - LSTM
    - Pre trained Chronos
    - Transformers
      - Unification
      - Non-Stationary Autoformer
      - BasisFormer
      - iTransformer
3. Results
4. Outlook
   - Chronos Simulation Framework
   - DYNOTEARS Causal Structure
   - Non linear causal structure
   - Data Augmentation

In [203]:
#!pip install torch==2.0.1

# 1. Data Loading & Preprocessing

In [204]:
import torch

In [205]:
import pandas as pd
import numpy as np

In [206]:
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.utils.weight_norm as wn
import torch.nn.functional as F

In [207]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import argparse
import os
import logging
import time
import importlib

In [208]:
#from torch.utils.tensorboard import SummaryWriter

In [209]:
##file_path = '/content/all_countries.csv' ## colab path
file_path = 'data/all_countries.csv' ## jupyter path
df = pd.read_csv(file_path)
df = pd.DataFrame(df)
df.head()

,Country,ISO3 Code,Datetime (UTC),Datetime (Local),Price (EUR/MWhe)
0,Austria,AUT,2015-01-01 00:00:00,2015-01-01 01:00:00,17.93
1,Austria,AUT,2015-01-01 01:00:00,2015-01-01 02:00:00,15.17
2,Austria,AUT,2015-01-01 02:00:00,2015-01-01 03:00:00,16.38
3,Austria,AUT,2015-01-01 03:00:00,2015-01-01 04:00:00,17.38
4,Austria,AUT,2015-01-01 04:00:00,2015-01-01 05:00:00,16.38


In [210]:
df = df [['Country','Datetime (UTC)',  'Price (EUR/MWhe)']]
df = df.pivot(index='Datetime (UTC)', columns='Country', values='Price (EUR/MWhe)')
df.head()

Country,Austria,Belgium,Bulgaria,Croatia,Czechia,Denmark,Estonia,Finland,France,Germany,...,Norway,Poland,Portugal,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland
Datetime (UTC),,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,17.93,34.94,NaN,NaN,24.20,18.29,23.37,23.37,34.94,17.93,...,27.36,17.18,48.10,44.17,NaN,24.20,23.25,48.10,23.37,43.43
2015-01-01 01:00:00,15.17,32.19,NaN,NaN,22.06,16.04,19.33,19.33,32.19,15.17,...,27.24,17.38,47.33,39.17,NaN,22.06,22.20,47.33,19.33,38.08
2015-01-01 02:00:00,16.38,28.05,NaN,NaN,20.27,14.60,17.66,17.66,23.53,16.38,...,27.16,17.40,42.27,26.93,NaN,20.27,19.56,42.27,17.66,35.47
2015-01-01 03:00:00,17.38,28.04,NaN,NaN,19.17,14.95,17.53,17.53,22.92,17.38,...,27.15,18.60,38.41,20.94,NaN,19.17,18.88,38.41,17.53,30.83
2015-01-01 04:00:00,16.38,34.26,NaN,NaN,17.90,14.50,18.07,18.07,34.26,16.38,...,27.30,19.30,35.72,18.52,NaN,17.90,18.39,35.72,18.07,28.26


In [211]:
print(df.isnull().sum())

Country
Austria                0
Belgium                0
Bulgaria           15336
Croatia            24096
Czechia                0
Denmark                0
Estonia                0
Finland                0
France                 0
Germany                0
Greece                 0
Hungary                0
Ireland            12480
Italy                  0
Latvia                 0
Lithuania              0
Luxembourg             0
Netherlands            0
North Macedonia    73008
Norway                 0
Poland                 0
Portugal               0
Romania                0
Serbia             16800
Slovakia               0
Slovenia               0
Spain                  0
Sweden                 0
Switzerland            0
dtype: int64


In [212]:
df = df.dropna(axis=1)
print(df.isnull().sum())

Country
Austria        0
Belgium        0
Czechia        0
Denmark        0
Estonia        0
Finland        0
France         0
Germany        0
Greece         0
Hungary        0
Italy          0
Latvia         0
Lithuania      0
Luxembourg     0
Netherlands    0
Norway         0
Poland         0
Portugal       0
Romania        0
Slovakia       0
Slovenia       0
Spain          0
Sweden         0
Switzerland    0
dtype: int64


In [213]:
df.reset_index(inplace=True)
df.columns.name = None
print(df.head())

        Datetime (UTC)  Austria  Belgium  Czechia  Denmark  Estonia  Finland  \
0  2015-01-01 00:00:00    17.93    34.94    24.20    18.29    23.37    23.37   
1  2015-01-01 01:00:00    15.17    32.19    22.06    16.04    19.33    19.33   
2  2015-01-01 02:00:00    16.38    28.05    20.27    14.60    17.66    17.66   
3  2015-01-01 03:00:00    17.38    28.04    19.17    14.95    17.53    17.53   
4  2015-01-01 04:00:00    16.38    34.26    17.90    14.50    18.07    18.07   

   France  Germany  Greece  ...  Netherlands  Norway  Poland  Portugal  \
0   34.94    17.93   48.78  ...        34.94   27.36   17.18     48.10   
1   32.19    15.17   31.10  ...        32.19   27.24   17.38     47.33   
2   23.53    16.38   20.78  ...        28.05   27.16   17.40     42.27   
3   22.92    17.38   25.40  ...        28.04   27.15   18.60     38.41   
4   34.26    16.38   26.00  ...        34.26   27.30   19.30     35.72   

   Romania  Slovakia  Slovenia  Spain  Sweden  Switzerland  
0    44.17   

In [214]:
df['Datetime (UTC)'] = pd.to_datetime(df['Datetime (UTC)'])
last_time_point = df['Datetime (UTC)'].max()
print("Last time point available:", last_time_point)

Last time point available: 2024-03-31 23:00:00


In [215]:
# Find the latest timestamp in the DataFrame
latest_timestamp = df['Datetime (UTC)'].max()

# Calculate the timestamp for 2500 hours before the latest timestamp
start_timestamp = latest_timestamp - pd.Timedelta(hours=2500)

# Filter the DataFrame for the last 2500 hours
df = df[df['Datetime (UTC)'] >= start_timestamp]
print(df)

           Datetime (UTC)  Austria  Belgium  Czechia  Denmark  Estonia  \
78571 2023-12-18 19:00:00    89.00    81.68    91.54    37.53    85.79   
78572 2023-12-18 20:00:00    84.92    79.97    76.34    33.87    76.89   
78573 2023-12-18 21:00:00    72.05    73.79    63.56    27.91    12.66   
78574 2023-12-18 22:00:00    59.58    68.67    53.21    31.94    10.08   
78575 2023-12-18 23:00:00    64.79    65.61    63.10    24.84    10.04   
...                   ...      ...      ...      ...      ...      ...   
81067 2024-03-31 19:00:00    66.17    47.01    68.37    70.00    50.09   
81068 2024-03-31 20:00:00    61.25    43.70    63.26    64.51    46.28   
81069 2024-03-31 21:00:00    44.99    50.29    51.29    54.90    43.98   
81070 2024-03-31 22:00:00    40.70    50.32    46.39    49.95    40.41   
81071 2024-03-31 23:00:00    32.10    44.39    42.60    48.98    40.39   

       Finland  France  Germany  Greece  ...  Netherlands  Norway  Poland  \
78571    17.20   82.07    77.98  1

In [216]:
df['month'] = df['Datetime (UTC)'].apply(lambda row: row.month)
df['day'] = df['Datetime (UTC)'].apply(lambda row: row.day)
df['weekday'] = df['Datetime (UTC)'].apply(lambda row: row.weekday())
df['hour'] = df['Datetime (UTC)'].apply(lambda row: row.hour)

print(df.head())

           Datetime (UTC)  Austria  Belgium  Czechia  Denmark  Estonia  \
78571 2023-12-18 19:00:00    89.00    81.68    91.54    37.53    85.79   
78572 2023-12-18 20:00:00    84.92    79.97    76.34    33.87    76.89   
78573 2023-12-18 21:00:00    72.05    73.79    63.56    27.91    12.66   
78574 2023-12-18 22:00:00    59.58    68.67    53.21    31.94    10.08   
78575 2023-12-18 23:00:00    64.79    65.61    63.10    24.84    10.04   

       Finland  France  Germany  Greece  ...  Romania  Slovakia  Slovenia  \
78571    17.20   82.07    77.98  121.73  ...   121.73     79.23     97.78   
78572    15.21   77.93    71.09  101.76  ...   101.76    105.45     89.42   
78573    12.66   83.20    62.98  108.33  ...    93.64     66.12     79.61   
78574    10.08   79.46    55.12  102.52  ...    57.20     55.42     61.41   
78575     8.06   78.51    47.29   91.55  ...    72.64     55.56     69.50   

        Spain  Sweden  Switzerland  month  day  weekday  hour  
78571  174.00   17.20       

In [217]:
# separating the electricity prices and timestamp features
electricity_prices_df = df[['Datetime (UTC)', 'Austria', 'Belgium', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France',
              'Germany', 'Greece', 'Hungary', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
             'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia',
             'Slovenia', 'Spain', 'Sweden', 'Switzerland']]
timestamp_features_df = df[['Datetime (UTC)', 'month', 'day', 'weekday', 'hour']]

# defining the split ratio
train_size = 0.8
train_size_electricity = int(len(electricity_prices_df) * train_size)
train_size_timestamp = int(len(timestamp_features_df) * train_size)

# spliting the data into train and test sets
electricity_prices_train = electricity_prices_df[:train_size_electricity]
electricity_prices_test = electricity_prices_df[train_size_electricity:]
timestamp_features_train = timestamp_features_df[:train_size_timestamp]
timestamp_features_test = timestamp_features_df[train_size_timestamp:]

In [218]:
#print(electricity_prices_train.head())

In [219]:
#country_names = electricity_prices_train.drop(columns=['Datetime (UTC)']).columns.tolist()

In [220]:
# rescaling the electricity prices
#scaler = StandardScaler()
scaler = MinMaxScaler()


electricity_prices_train_scaled = scaler.fit_transform(electricity_prices_train.drop(columns=['Datetime (UTC)']))
electricity_prices_test_scaled = scaler.transform(electricity_prices_test.drop(columns=['Datetime (UTC)']))

In [221]:
zero_count = np.sum(electricity_prices_train_scaled == 0)
print(f"Number of zero values in actual data: {zero_count}")

Number of zero values in actual data: 222


In [222]:
#electricity_prices_train_scaled

In [223]:
def create_sequences(data, seq_length, pred_length, label_length, curr_model):
    seq_x = [] # storing for input seqiences
    seq_y = [] # storing for output seqiences
    for i in range(len(data) - seq_length - pred_length):
        seq_x.append(data[i:i+seq_length])
        if curr_model in ["basis_former", "itransformer", "ns_autoformer"]:
          seq_y.append(data[i+seq_length-label_length:i+seq_length+pred_length])
        else: ## only chronos
          seq_y.append(data[i+seq_length:i+seq_length+pred_length])
    return np.array(seq_x), np.array(seq_y)

In [224]:
def create_dataloader(seq_x, seq_y, seq_x_mark, seq_y_mark, batch_size, curr_model):
    seq_x = torch.tensor(seq_x, dtype=torch.float32)
    seq_y = torch.tensor(seq_y, dtype=torch.float32)
    seq_x_mark = torch.tensor(seq_x_mark, dtype=torch.float32)
    seq_y_mark = torch.tensor(seq_y_mark, dtype=torch.float32)
    
    if curr_model == "basis_former":
        indices = []
        total_len = len(seq_x)
        for i in range(total_len):
            index_list = np.arange(i, i + len(seq_x[0]) + len(seq_y[0]), 1)
            norm_index = index_list / total_len
            indices.append(norm_index)
        indices = torch.tensor(indices, dtype=torch.float32)
        dataset = TensorDataset(seq_x, seq_y, seq_x_mark, seq_y_mark, indices)
    else:
        dataset = TensorDataset(seq_x, seq_y, seq_x_mark, seq_y_mark)
    
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=2, shuffle=True, drop_last=True)
    return dataloader

# 2. Experimental Design

In [225]:
seq_length = 96
pred_length = 48
label_length = 48
batch_size = 24
#device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")

# Benchmark Models

## Linear Regression

### Linear Model Preprocessing

In [226]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Copy the main dataframe for linear model preprocessing
df_linear = df.copy()

# Define the number of lagged features
lag_steps = 3

# List of all columns (already defined in the main preprocessing)
all_columns = df_linear.columns.tolist()

# List of columns to exclude (non-country columns)
exclude_columns = ['Datetime (UTC)', 'month', 'day', 'weekday', 'hour']

# Define the country columns by excluding non-country columns
countries = [col for col in all_columns if col not in exclude_columns]

# Create lagged features for each country
for country in countries:
    for lag in range(1, lag_steps + 1):
        df_linear[f'{country}_lag_{lag}'] = df_linear[country].shift(lag)

# Drop rows with NaN values due to lagging
df_linear.dropna(inplace=True)

# Define features (X) and targets (Y)
X_numerical = df_linear.drop(columns=countries + ['Datetime (UTC)'])
Y = df_linear[countries]  # Target: current prices of all countries

# Standardize the features and targets
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X_numerical)

scaler_Y = StandardScaler()
Y_scaled = scaler_Y.fit_transform(Y)

# Split data into training and testing sets
train_size = 0.8
train_size_idx = int(len(X_scaled) * train_size)
X_train, X_test = X_scaled[:train_size_idx], X_scaled[train_size_idx:]
Y_train, Y_test = Y_scaled[:train_size_idx], Y_scaled[train_size_idx:]

### Linear Regression Model

In [227]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Initialize and train the linear regression model
model_lr = LinearRegression()
model_lr.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred_lr = model_lr.predict(X_test)

# Inverse transform the predictions and the actual values back to the original scale
Y_pred_lr_original = scaler_Y.inverse_transform(Y_pred_lr)
Y_test_original = scaler_Y.inverse_transform(Y_test)

# MAE
mae_lr = mean_absolute_error(Y_test_original, Y_pred_lr_original)
print(f"Linear Regression Mean Absolute Error: {mae_lr}")

# Evaluate the model using RMSE
rmse_lr = np.sqrt(mean_squared_error(Y_test_original, Y_pred_lr_original))
print(f"Linear Regression Root Mean Squared Error: {rmse_lr}")

Linear Regression Mean Absolute Error: 7.121460465848678
Linear Regression Root Mean Squared Error: 12.298510526240207


## LSTM

### LSTM Preprocessing

In [228]:
from sklearn.preprocessing import StandardScaler

# Copy the main dataframe for LSTM model preprocessing
df_lstm = df.copy()

# Rescale the data using StandardScaler for LSTM
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_lstm.drop(columns=['Datetime (UTC)', 'month', 'day', 'weekday', 'hour']))

# Convert to a supervised learning problem by creating sequences
def create_sequences_lstm(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Define sequence length (number of time steps)
seq_length = 24

# Create sequences
X, y = create_sequences_lstm(data_scaled, seq_length)

# Split data into training and testing sets
train_size = int(X.shape[0] * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

### LSTM Model

In [229]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, X.shape[2])))
model_lstm.add(LSTM(units=50, return_sequences=False))
model_lstm.add(Dense(units=y.shape[1]))

# Compile the model
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Set up EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fit the model with EarlyStopping
model_lstm.fit(X_train, y_train, epochs=50, batch_size=24, 
               validation_data=(X_test, y_test), callbacks=[early_stopping])

# Make predictions
y_pred_lstm = model_lstm.predict(X_test)

# Inverse transform the scaled data to original values
y_test_inverse = scaler.inverse_transform(y_test)
y_pred_inverse = scaler.inverse_transform(y_pred_lstm)

# Evaluate the model using MAE
mae_lstm = mean_absolute_error(y_test_inverse, y_pred_inverse)
print(f"LSTM Mean Absolute Error: {mae_lstm}")

# Evaluate the model using RMSE
rmse_lstm = np.sqrt(mean_squared_error(y_test_inverse, y_pred_inverse))
print(f"LSTM Root Mean Squared Error: {rmse_lstm}")

Epoch 1/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.9136 - val_loss: 0.5468
Epoch 2/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.4562 - val_loss: 0.4291
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.3066 - val_loss: 0.3136
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.2233 - val_loss: 0.2464
Epoch 5/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1892 - val_loss: 0.2221
Epoch 6/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.1654 - val_loss: 0.2072
Epoch 7/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.1273 - val_loss: 0.2002
Epoch 8/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.1243 - val_loss: 0.1920
Epoch 9/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1132 - val_loss: 0.1896
Epoch 10/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0961 - val_loss: 0.1798
Epoch 11/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0965 - val_loss: 0.1735
Epoch 12/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0

# Pre trained model Chronos

zero shot evaluation with Chronos Pretrained Model

In [ ]:
#!pip install git+https://github.com/amazon-science/chronos-forecasting.git

In [59]:
import chronos
import pandas as pd
import numpy as np
import torch
from chronos import ChronosPipeline
import matplotlib.pyplot as plt

# Assuming df is your initial dataset
# Make sure 'Datetime (UTC)' is set as the index
#df = df.set_index('Datetime (UTC)')

# Select the countries we want to forecast
selected_countries = ['Austria', 'Belgium', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France',
              'Germany', 'Greece', 'Hungary', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
             'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia',
             'Slovenia', 'Spain', 'Sweden', 'Switzerland']

# Extract the data for selected countries
data = df[selected_countries]

# Print some basic information about the dataset
#print(f"Dataset shape: {data.shape}")
#print(f"Date range: from {data.index.min()} to {data.index.max()}")
#print(f"Any missing values: {data.isnull().any().any()}")

# Define the split point for train and test
split_point = int(len(data) * 0.8)  # 80% for training, 20% for testing

# Split the data
train_data = data.iloc[:split_point]
test_data = data.iloc[split_point:]

#print(f"\nTrain data shape: {train_data.shape}")
#print(f"Test data shape: {test_data.shape}")

# Function to prepare data for Chronos
def prepare_chronos_data(data, seq_length):
    return torch.tensor(data[-seq_length:].values, dtype=torch.float32).unsqueeze(0)

# Initialize Chronos pipeline
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cpu",  # use "cpu" for CPU inference and "mps" for Apple Silicon
    torch_dtype=torch.bfloat16,
)

# Set parameters
seq_length = 96
pred_length = 48

#print(f"\nSequence length: {seq_length} (equivalent to {seq_length/24} days)")
#print(f"Prediction length: {pred_length} (equivalent to {pred_length/24} days)")

# Prepare data and run forecasts
results = {}

for country in selected_countries:
    print(f"\nProcessing {country}")
    print(f"Data range for {country}: {train_data[country].min()} to {train_data[country].max()}")
    
    # Prepare input for Chronos
    chronos_input = prepare_chronos_data(train_data[country], seq_length)
    print(f"Chronos input shape: {chronos_input.shape}")
    
    # Generate forecast
    forecast = pipeline.predict(
        context=chronos_input,
        prediction_length=pred_length,
        num_samples=20,
    )
    
    # Store results
    results[country] = forecast
    print(f"Forecast shape: {forecast.shape}")


# Evaluate results
def calculate_mse(actual, forecast):
    return np.mean((actual - forecast) ** 2)
    
def calculate_mae(actual, forecast):
    return np.mean(np.abs(actual - forecast))
    
def calculate_rmse(actual, forecast):
    return np.sqrt(calculate_mse(forecast, actual))
    
def calculate_mape(actual, forecast):
    return np.mean(np.abs((forecast - actual) / actual))

mse_results = {}
mae_results = {}
mape_results = {}
rmse_results = {}

for country in selected_countries:
    actual_values = test_data[country].values[:pred_length]
    forecasted_values = np.median(results[country], axis=1).flatten()
    
    min_length = min(len(actual_values), len(forecasted_values))
    actual_values = actual_values[:min_length]
    forecasted_values = forecasted_values[:min_length]
    
    mse = calculate_mse(actual_values, forecasted_values)
    mae = calculate_mae(actual_values, forecasted_values)
    rmse = calculate_rmse(actual_values, forecasted_values)
    mape = calculate_mape(actual_values, forecasted_values)
    mse_results[country] = mse
    mae_results[country] = mae
    rmse_results[country] = rmse
    mape_results[country] = mape
    print(f"\nMSE for {country}: {mse}")
    print(f"MAE for {country}: {mae}")
    print(f"\nRMSE for {country}: {rmse}")
    print(f"MAPE for {country}: {mape}")
    
    # Debug information
    #print(f"Actual values shape: {actual_values.shape}")
    #print(f"Forecasted values shape: {forecasted_values.shape}")
    #print(f"Actual values range: {actual_values.min()} to {actual_values.max()}")
    #print(f"Forecasted values range: {forecasted_values.min()} to {forecasted_values.max()}")
    
    # Plot actual vs predicted
    #plt.figure(figsize=(12, 6))
    
    # Plot actual values
    #if len(actual_values) == 1:
    #    plt.scatter(0, actual_values[0], label='Actual', marker='o', s=100, color='blue')
    #else:
    #    plt.plot(actual_values, label='Actual', marker='o')
    
    # Plot forecasted values
    #plt.plot(forecasted_values, label='Predicted', marker='x', color='red')
    
    #plt.title(f'{country} - Actual vs Predicted')
    #plt.xlabel('Time Steps')
    #plt.ylabel('Price')
    #plt.legend()
    #plt.grid(True)
    
    # Add text annotations
    #if len(actual_values) > 0:
    #    plt.annotate(f'{actual_values[0]:.2f}', (0, actual_values[0]), textcoords="offset points", xytext=(0,10), ha='center')
    #if len(actual_values) > 1:
    #    plt.annotate(f'{actual_values[-1]:.2f}', (len(actual_values)-1, actual_values[-1]), textcoords="offset points", xytext=(0,10), ha='center')
    
    #plt.annotate(f'{forecasted_values[0]:.2f}', (0, forecasted_values[0]), textcoords="offset points", xytext=(0,-15), ha='center')
    #plt.annotate(f'{forecasted_values[-1]:.2f}', (len(forecasted_values)-1, forecasted_values[-1]), textcoords="offset points", xytext=(0,-15), ha='center')
    
    #plt.tight_layout()
    #plt.show()

# Calculate and print average MSE
average_mse = np.mean(list(mse_results.values()))
print(f"\nAverage MSE across selected countries: {average_mse}")
average_mae = np.mean(list(mae_results.values()))
print(f"\nAverage MAE across selected countries: {average_mae}")
average_rmse = np.mean(list(rmse_results.values()))
print(f"\nAverage RMSE across selected countries: {average_rmse}")
average_mape = np.mean(list(mape_results.values()))
print(f"\nAverage MAPE across selected countries: {average_mape}")

# Print summary statistics
#for country in selected_countries:
    #print(f"\nSummary for {country}:")
    #print(f"Train data mean: {train_data[country].mean():.2f}, std: {train_data[country].std():.2f}")
    #print(f"Test data mean: {test_data[country].mean():.2f}, std: {test_data[country].std():.2f}")
    #print(f"MSE: {mse_results[country]:.2f}")
    #print(f"RMSE: {np.sqrt(mse_results[country]):.2f}")
    #print(f"MAE: {mae_results[country]:.2f}")
    #print(f"RMSE as percentage of mean: {(np.sqrt(mse_results[country]) / test_data[country].mean()) * 100:.2f}%")
    #print(f"MAE as percentage of mean: {(mae_results[country] / test_data[country].mean()) * 100:.2f}%")

# Save MSE results
#import pickle
#with open('error_results_chronos_unscaled.pkl', 'wb') as f:
    #pickle.dump({'mse': mse_results, 'mae': mae_results}, f)

/home/RDC/inceemir/apa_group4_transformers_for_multivar_energy_forecasting/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Processing Austria
Data range for Austria: -12.5 to 147.04
Chronos input shape: torch.Size([1, 96])
Forecast shape: torch.Size([1, 20, 48])

Processing Belgium
Data range for Belgium: -11.8 to 144.11
Chronos input shape: torch.Size([1, 96])
Forecast shape: torch.Size([1, 20, 48])

Processing Czechia
Data range for Czechia: -17.09 to 158.44
Chronos input shape: torch.Size([1, 96])
Forecast shape: torch.Size([1, 20, 48])

Processing Denmark
Data range for Denmark: -4.97 to 275.85
Chronos input shape: torch.Size([1, 96])
Forecast shape: torch.Size([1, 20, 48])

Processing Estonia
Data range for Estonia: -2.02 to 1896.0
Chronos input shape: torch.Size([1, 96])
Forecast shape: torch.Size([1, 20, 48])

Processing Finland
Data range for Finland: -2.5 to 1896.0
Chronos input shape: torch.Size([1, 96])
Forecast shape: torch.Size([1, 20, 48])

Processing France
Data range for France: -11.93 to 144.11
Chronos input shape: torch.Size([1, 96])
Forecast shape: torch.Size([1, 20, 48])

Processing Ge

/tmp/ipykernel_1323976/618574630.py:88: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((forecast - actual) / actual))


In [64]:
rmse_chronos = average_rmse
print(rmse_chronos)

33.2765369406757


# Transformers

## Unification

In [230]:
def fit (model, train_flag, test_flag, train_loader=None, test_loader=None, pretrained_model=None):
    '''Fits a transformer model to the train and/or test loaders
    
    model - "basis_former", "itransformer", "ns_autoformer"
    
    train_flag: typ(bool) - True: to train the model on train_loader, False: if pretrained_model is passed
    
    test_flag: typ(bool) - True: to test on test_loader, False: if only training
    
    pretrained_model - pass a pretrained model if available to be fitted on a test_loader. 
    eg. fit(basis_former, train_flag=False, test_flag=True, test_loader=test_loader, pretrained_model=model)
    '''
    
    if curr_model == 'basis_former':
        # Code for Basisforme

        import Basisformer.model
        importlib.reload(Basisformer.model)
        from Basisformer.model import Basisformer

        import Basisformer.main
        importlib.reload(Basisformer.main)
        from Basisformer.main import parse_args, model_setup, log_and_print
        importlib.reload(Basisformer.pyplot)

        class Args:
            is_training = True
            data_path = 'data'
            device = 0
            num_workers = 10
            features = 'M'
            freq = 'h'
            seq_len = 96
            pred_len = 48
            heads = 16
            d_model = 512
            N = 10
            block_nums = 2
            bottleneck = 2
            map_bottleneck = 20
            train_epochs = 50
            batch_size = 24
            learning_rate = 0.0001
            tau = 0.07
            loss_weight_prediction = 1.0
            loss_weight_infonce = 1.0
            loss_weight_smooth = 1.0
            check_point = 'checkpoint'
            patience = 3

        args = Args()
        
        #args = parse_args()

        # Set up device
        device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")

        # Set up model
        model = model_setup(args, device)
        
        if pretrained_model == None:
            # Set up model
            model = model_setup(args, device)

        else:
            model = pretrained_model

        # Log arguments and model
        ##log_and_print('Args in experiment:')
        ##log_and_print(args)
        ##log_and_print(model)
        
        if train_flag:
            import Basisformer.model
            importlib.reload(Basisformer.model)
            from Basisformer.model import Basisformer

            import Basisformer.main
            importlib.reload(Basisformer.main)
            from Basisformer.main import train


            record_dir = os.path.join('records', args.data_path.split('.')[0], 'features_' + args.features,
                                    'seq_len' + str(args.seq_len) + ',' + 'pred_len' + str(args.pred_len))
            
            if train_loader == None:
                return 'train_loader not found'

            # Call the train function
            train(model, train_loader, args, device, record_dir)
            
        else:
            if pretrained_model == None:
                return 'model not found which is required for testing'
            
        if test_flag :
            import Basisformer.main
            importlib.reload(Basisformer.main)
            from Basisformer.main import test
            
            if test_loader == None:
                return 'test_loader not found'

            test(model, test_loader, args, device, record_dir)
        return model
            
    
    elif curr_model == 'itransformer':
        # code for itransformer
        
        import iTransformer.experiment
        importlib.reload(iTransformer.experiment)
        from iTransformer.experiment import Exp_Long_Term_Forecast
        
        class Args:
            is_training = 1
            model_id = 'iTransformer_train'
            model = 'iTransformer'
            data = 'all_countries'
            features = 'M'
            target = 'OT'
            freq = 'h'
            checkpoints = './checkpoints/'
            seq_len = 96
            label_len = 48
            pred_len = 48
            enc_in = 24
            dec_in = 24
            c_out = 24
            d_model = 512
            n_heads = 8
            e_layers = 2
            d_layers = 1
            d_ff = 2048
            moving_avg = 25
            factor = 1
            distil = True
            dropout = 0.05
            embed = 'timeF'
            activation = 'gelu'
            output_attention = False
            do_predict = True
            num_workers = 10
            itr = 2
            train_epochs = 50
            batch_size = 24
            patience = 3
            learning_rate = 0.0001
            des = 'test'
            loss = 'mse'
            lradj = 'type1'
            use_amp = False
            use_gpu = True if torch.cuda.is_available() else False
            gpu = 0
            use_multi_gpu = False
            devices = '0,1,2,3'
            exp_name = 'MTSF'
            channel_independence = False
            inverse = False
            class_strategy = 'projection'
            target_root_path = './data'
            target_data_path = 'all_countries'
            efficient_training = False
            use_norm = True
            partial_start_index = 0
            seed = 2021
            p_hidden_dims = [128, 128]
            p_hidden_layers = 2

        args = Args()
        
        if pretrained_model == None:
            # Initialize the experiment
            exp = Exp_Long_Term_Forecast(args)

        else:
            return 'pretrained not valid for iTransformer and ns_autoformer'

        # Define the settings
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}'.format(
            args.model_id, args.model, args.data, args.features, args.seq_len, args.label_len,
            args.pred_len, args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff,
            args.factor, args.embed, args.distil, args.des, 0)
        
        if train_flag:
            Exp_Long_Term_Forecast.train(self=exp, train_loader=train_loader, setting=setting)
        
        if test_flag:
            Exp_Long_Term_Forecast.test(self=exp, test_loader=test_loader, setting=setting, test=0)
        return exp.model
    
    elif curr_model == 'ns_autoformer':
        # code for itransformer
        import ns_Autoformer.ns_Autoformer
        importlib.reload(ns_Autoformer.ns_Autoformer)
        from ns_Autoformer.ns_Autoformer import Model

        # import ns_Autoformer.main
        # importlib.reload(ns_Autoformer.main)
        # from ns_Autoformer.main import parse_args
        
        from ns_Autoformer.main import Exp_Main

        class Args:
            is_training = 1
            model_id = 'ns_autoformer_train'
            model = 'ns_Autoformer'
            features = 'M'
            target = 'OT'
            freq = 'h'
            checkpoints = './checkpoints/'
            seq_len = 96
            label_len = 48
            pred_len = 48
            enc_in = 24
            dec_in = 24
            c_out = 24
            d_model = 512
            n_heads = 8
            e_layers = 2
            d_layers = 1
            d_ff = 2048
            moving_avg = 25
            factor = 1
            distil = True
            dropout = 0.05
            embed = 'timeF'
            activation = 'gelu'
            output_attention = False
            do_predict = True
            num_workers = 10
            itr = 2
            train_epochs = 50
            batch_size = 24
            patience = 3
            learning_rate = 0.0001
            des = 'test'
            loss = 'mse'
            lradj = 'type1'
            use_amp = False
            use_gpu = True if torch.cuda.is_available() else False
            gpu = 0
            use_multi_gpu = False
            devices = '0,1,2,3'
            seed = 2021
            p_hidden_dims = [128, 128]
            p_hidden_layers = 2

        args = Args()

        # if args.use_gpu:
        #     if args.use_multi_gpu:
        #         args.devices = args.devices.replace(' ', '')
        #         device_ids = args.devices.split(',')
        #         args.device_ids = [int(id_) for id_ in device_ids]
        #         args.gpu = args.device_ids[0]
        #     else:
        #         torch.cuda.set_device(args.gpu)

        # print('Args in experiment:')
        # print(args)
        
        if pretrained_model == None:
            # Initialize the experiment
            exp = Exp_Main(args)

        # Define the setting string
        setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}'.format(
            args.model_id, args.model, args.features, args.seq_len, args.label_len,
            args.pred_len, args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff,
            args.factor, args.embed, args.distil, args.des, 0)
        
        if train_flag:
            Exp_Main.train(self=exp, train_loader=train_loader, setting=setting)
        
        if test_flag:
            Exp_Main.test(self=exp, test_loader=test_loader, setting=setting, test=0)
        return exp.model

## Basisformer

In [231]:
seq_length = 96
pred_length = 48
label_length = 48
curr_model = "basis_former"

train_seq_x, train_seq_y = create_sequences(electricity_prices_train_scaled, seq_length, pred_length, label_length, curr_model)
train_seq_x_mark, train_seq_y_mark = create_sequences(timestamp_features_train.drop(columns=['Datetime (UTC)']).values, seq_length, pred_length, label_length, curr_model)


test_seq_x, test_seq_y = create_sequences(electricity_prices_test_scaled, seq_length, pred_length,label_length, curr_model)
test_seq_x_mark, test_seq_y_mark = create_sequences(timestamp_features_test.drop(columns=['Datetime (UTC)']).values, seq_length, pred_length, label_length, curr_model)


In [232]:
print("Sample training sequence x:", train_seq_x[0])
print("Sample training sequence y:", train_seq_y[0])
print("Sample training sequence x mark:", train_seq_x_mark[0])
print("Sample training sequence y mark:", train_seq_y_mark[0])

Sample training sequence x: [[0.63620409 0.59957668 0.61886857 ... 0.95994704 0.03725768 0.65732351]
 [0.61063056 0.58860881 0.53227369 ... 0.71427783 0.03349409 0.62297112]
 [0.52996114 0.54897056 0.45946562 ... 0.60686307 0.02867139 0.58946614]
 ...
 [0.39632694 0.40985184 0.37440893 ... 0.3586009  0.08359338 0.60902158]
 [0.41707409 0.43390418 0.39013274 ... 0.38894406 0.0790922  0.60771788]
 [0.38071957 0.38406773 0.35595055 ... 0.39710913 0.07313475 0.58229581]]
Sample training sequence y: [[0.572521   0.57706369 0.539338   ... 0.43236235 0.13401418 0.62831628]
 [0.50231917 0.50522737 0.4752464  ... 0.39644709 0.12514421 0.63489994]
 [0.46333208 0.46962991 0.44727397 ... 0.33885027 0.11810875 0.66905678]
 ...
 [0.08668672 0.13873388 0.0151541  ... 0.4279488  0.07952719 0.13010886]
 [0.08668672 0.14040151 0.0623825  ... 0.56024495 0.07232151 0.15155466]
 [0.06506205 0.13655314 0.05184299 ... 0.60300121 0.0661182  0.12541555]]
Sample training sequence x mark: [[12 18  0 19]
 [12 18 

In [233]:
batch_size = 24

# converting sequences to PyTorch DataLoader objects
train_loader = create_dataloader(train_seq_x, train_seq_y, train_seq_x_mark, train_seq_y_mark, batch_size, curr_model)
test_loader = create_dataloader(test_seq_x, test_seq_y, test_seq_x_mark, test_seq_y_mark, batch_size, curr_model)

In [234]:
for batch in train_loader:
    print(f"Batch contains {len(batch)} items:")
    for i, item in enumerate(batch):
        print(f"Item {i}: Shape = {item.shape if torch.is_tensor(item) else 'Not a tensor'}")
    break  # Just print the first batch

Batch contains 5 items:
Item 0: Shape = torch.Size([24, 96, 24])
Item 1: Shape = torch.Size([24, 96, 24])
Item 2: Shape = torch.Size([24, 96, 4])
Item 3: Shape = torch.Size([24, 96, 4])
Item 4: Shape = torch.Size([24, 192])


In [ ]:
##pip install adabelief_pytorch==0.2.1

### Train

In [235]:
%%capture captured_output
curr_model = "basis_former" 
basisformer_train_test = fit(model=curr_model, train_flag=True, test_flag=True, train_loader=train_loader, test_loader=test_loader)

INFO:root:	iters: 15, epoch: 1 | loss: 2.7772989
INFO:root:	iters: 30, epoch: 1 | loss: 1.0085220
INFO:root:	iters: 45, epoch: 1 | loss: 0.2749456
INFO:root:	iters: 60, epoch: 1 | loss: 0.1483428
INFO:root:	iters: 75, epoch: 1 | loss: 0.1595481
INFO:root:Epoch: 1 cost time: 3.381474494934082
INFO:root:loss_pred:0.016862093350039672
INFO:root:loss entropy:1.0131029602972292
INFO:root:loss smooth:0.13612646745009857
INFO:root:Epoch: 1 | Train Loss: 1.1660915
INFO:root:	iters: 15, epoch: 2 | loss: 0.1459127
INFO:root:	iters: 30, epoch: 2 | loss: 0.1475278
INFO:root:	iters: 45, epoch: 2 | loss: 0.1204328
INFO:root:	iters: 60, epoch: 2 | loss: 0.1706662
INFO:root:	iters: 75, epoch: 2 | loss: 0.2184069
INFO:root:Epoch: 2 cost time: 3.4860928058624268
INFO:root:loss_pred:0.01676437042218137
INFO:root:loss entropy:0.05144979293713392
INFO:root:loss smooth:0.10698402421428012
INFO:root:Epoch: 2 | Train Loss: 0.1751982
INFO:root:	iters: 15, epoch: 3 | loss: 0.1885138
INFO:root:	iters: 30, epoch:

## iTransformer

In [236]:
seq_length = 96
pred_length = 48
label_length = 48
curr_model = "itransformer"

train_seq_x, train_seq_y = create_sequences(electricity_prices_train_scaled, seq_length, pred_length, label_length, curr_model)
train_seq_x_mark, train_seq_y_mark = create_sequences(timestamp_features_train.drop(columns=['Datetime (UTC)']).values, seq_length, pred_length, label_length, curr_model)


test_seq_x, test_seq_y = create_sequences(electricity_prices_test_scaled, seq_length, pred_length,label_length, curr_model)
test_seq_x_mark, test_seq_y_mark = create_sequences(timestamp_features_test.drop(columns=['Datetime (UTC)']).values, seq_length, pred_length, label_length, curr_model)


# converting sequences to PyTorch DataLoader objects
train_loader = create_dataloader(train_seq_x, train_seq_y, train_seq_x_mark, train_seq_y_mark, batch_size, curr_model)
test_loader = create_dataloader(test_seq_x, test_seq_y, test_seq_x_mark, test_seq_y_mark, batch_size, curr_model)

### Train

In [237]:
itransformer_train_test = fit(model=curr_model, train_flag=True, test_flag=True, train_loader=train_loader, test_loader=test_loader)

Use GPU: cuda:0
Epoch: 1 cost time: 0.9888520240783691
Epoch: 1, Steps: 77 | Train Loss: 0.0118243
Validation loss decreased (inf --> 0.011824).  Saving model ...
Updating learning rate to 0.0001
Epoch: 2 cost time: 1.0110671520233154
Epoch: 2, Steps: 77 | Train Loss: 0.0095816
Validation loss decreased (0.011824 --> 0.009582).  Saving model ...
Updating learning rate to 5e-05
Epoch: 3 cost time: 1.0544493198394775
Epoch: 3, Steps: 77 | Train Loss: 0.0087091
Validation loss decreased (0.009582 --> 0.008709).  Saving model ...
Updating learning rate to 2.5e-05
Epoch: 4 cost time: 1.0203924179077148
Epoch: 4, Steps: 77 | Train Loss: 0.0080874
Validation loss decreased (0.008709 --> 0.008087).  Saving model ...
Updating learning rate to 1.25e-05
Epoch: 5 cost time: 0.9425961971282959
Epoch: 5, Steps: 77 | Train Loss: 0.0077332
Validation loss decreased (0.008087 --> 0.007733).  Saving model ...
Updating learning rate to 6.25e-06
Epoch: 6 cost time: 0.9534096717834473
Epoch: 6, Steps: 77 |

## Nonstationary Transformer

In [238]:
curr_model = "ns_autoformer"

train_seq_x, train_seq_y = create_sequences(electricity_prices_train_scaled, seq_length, pred_length, label_length, curr_model)
train_seq_x_mark, train_seq_y_mark = create_sequences(timestamp_features_train.drop(columns=['Datetime (UTC)']).values, seq_length, pred_length, label_length, curr_model)


test_seq_x, test_seq_y = create_sequences(electricity_prices_test_scaled, seq_length, pred_length,label_length, curr_model)
test_seq_x_mark, test_seq_y_mark = create_sequences(timestamp_features_test.drop(columns=['Datetime (UTC)']).values, seq_length, pred_length, label_length, curr_model)


# converting sequences to PyTorch DataLoader objects
train_loader = create_dataloader(train_seq_x, train_seq_y, train_seq_x_mark, train_seq_y_mark, batch_size, curr_model)
test_loader = create_dataloader(test_seq_x, test_seq_y, test_seq_x_mark, test_seq_y_mark, batch_size, curr_model)

### Train

In [239]:
ns_autoformer_train_test = fit(model=curr_model, train_flag=True, test_flag=True, train_loader=train_loader, test_loader=test_loader)

Use GPU: cuda:0
Epoch: 1 cost time: 3.612320899963379
Epoch: 1, Steps: 77 | Train Loss: 0.0235922
Validation loss decreased (inf --> 0.023592).  Saving model ...
Updating learning rate to 0.0001
Epoch: 2 cost time: 3.5533363819122314
Epoch: 2, Steps: 77 | Train Loss: 0.0137995
Validation loss decreased (0.023592 --> 0.013800).  Saving model ...
Updating learning rate to 5e-05
Epoch: 3 cost time: 3.57230806350708
Epoch: 3, Steps: 77 | Train Loss: 0.0116421
Validation loss decreased (0.013800 --> 0.011642).  Saving model ...
Updating learning rate to 2.5e-05
Epoch: 4 cost time: 3.575103998184204
Epoch: 4, Steps: 77 | Train Loss: 0.0103056
Validation loss decreased (0.011642 --> 0.010306).  Saving model ...
Updating learning rate to 1.25e-05
Epoch: 5 cost time: 3.555845260620117
Epoch: 5, Steps: 77 | Train Loss: 0.0096618
Validation loss decreased (0.010306 --> 0.009662).  Saving model ...
Updating learning rate to 6.25e-06
Epoch: 6 cost time: 3.587740421295166
Epoch: 6, Steps: 77 | Train

# 3. Results - Models Performance Comparison

In [249]:
import pandas as pd

# Create a dictionary to hold the results
results = {
    'Model': ['Linear Regression', 'LSTM', 'Chronos', 'Nonstationary Autoformer', 'Basisformer', 'iTransformer'],
    # 'RMSE': [rmse_lr, rmse_lstm, rmse_chronos, rmse_nsautoformer, rmse_basisformer, rmse_itransformer]
    'RMSE': [12.29, 14.85, 33.28, 0.14, 0.15, 0.12]
}

# Convert the dictionary into a DataFrame
results_df = pd.DataFrame(results)

# Display the results table
print(results_df)

                      Model   RMSE
0         Linear Regression  12.29
1                      LSTM  14.85
2                   Chronos  33.28
3  Nonstationary Autoformer   0.14
4               Basisformer   0.15
5              iTransformer   0.12


# 4. Outlook 

## Chronos Simulation

In [22]:
!pip install "chronos[training] @ git+https://github.com/amazon-science/chronos-forecasting.git"

In [ ]:
!python supporting_files_chronos/kernel-synth.py --num-series 500 --max-kernels 2

In [242]:
import pyarrow.ipc as ipc

file_path = 'supporting_files_chronos/kernelsynth-data.arrow'

with open(file_path, 'rb') as f:
    reader = ipc.RecordBatchFileReader(f)
    table = reader.read_all()

df_ch = table.to_pandas()

print(df_ch)

         start target._np_shape  \
0   2000-01-01        [1024, 3]   
1   2000-01-01        [1024, 3]   
2   2000-01-01        [1024, 3]   
3   2000-01-01        [1024, 3]   
4   2000-01-01        [1024, 3]   
..         ...              ...   
495 2000-01-01        [1024, 3]   
496 2000-01-01        [1024, 3]   
497 2000-01-01        [1024, 3]   
498 2000-01-01        [1024, 3]   
499 2000-01-01        [1024, 3]   

                                                target  
0    [-1.3293273404902177e-07, 1.7617705479648852e-...  
1    [0.8820515850319665, -2.630758929416405, -0.28...  
2    [1.0917233174006584, -0.30767090449730106, 0.1...  
3    [-1.4419568333102197, -0.2799756872675115, -0....  
4    [-0.8402086553986396, -1.5435372346614247, -0....  
..                                                 ...  
495  [5.2983733022775885, -9.044641569560579, -3.15...  
496  [-0.7313010836303305, -0.8540684084420854, 0.9...  
497  [-0.8684795217377739, -0.5001586491794001, 0.2...  
498  [0.8

In [243]:
import matplotlib.pyplot as plt 

# Number of time series
num_series = 15
# Number of plots per row
plots_per_row = 5
# Number of rows
num_rows = (num_series + plots_per_row - 1) // plots_per_row

fig, axes = plt.subplots(num_rows, plots_per_row, figsize=(15, num_rows * 3))

for i in range(num_series):
    row = i // plots_per_row
    col = i % plots_per_row
    ax = axes[row, col]
    ax.plot(df_ch['target'].iloc[i])
    ax.set_title(f'Time Series {i}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')

# Remove any empty subplots
for j in range(i + 1, num_rows * plots_per_row):
    fig.delaxes(axes.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
!python supporting_files_chronos/kernel-synth-mult.py --num-series 500 --max-kernels 2 --dimensions 3

In [244]:
import pyarrow.ipc as ipc

file_path = 'supporting_files_chronos/kernelsynth-data.arrow'

with open(file_path, 'rb') as f:
    reader = ipc.RecordBatchFileReader(f)
    table = reader.read_all()

df_ch_mult = table.to_pandas()

print(df_ch_mult)

         start target._np_shape  \
0   2000-01-01        [1024, 3]   
1   2000-01-01        [1024, 3]   
2   2000-01-01        [1024, 3]   
3   2000-01-01        [1024, 3]   
4   2000-01-01        [1024, 3]   
..         ...              ...   
495 2000-01-01        [1024, 3]   
496 2000-01-01        [1024, 3]   
497 2000-01-01        [1024, 3]   
498 2000-01-01        [1024, 3]   
499 2000-01-01        [1024, 3]   

                                                target  
0    [-1.3293273404902177e-07, 1.7617705479648852e-...  
1    [0.8820515850319665, -2.630758929416405, -0.28...  
2    [1.0917233174006584, -0.30767090449730106, 0.1...  
3    [-1.4419568333102197, -0.2799756872675115, -0....  
4    [-0.8402086553986396, -1.5435372346614247, -0....  
..                                                 ...  
495  [5.2983733022775885, -9.044641569560579, -3.15...  
496  [-0.7313010836303305, -0.8540684084420854, 0.9...  
497  [-0.8684795217377739, -0.5001586491794001, 0.2...  
498  [0.8

In [245]:
print(df_ch_mult.head())
print(df_ch_mult['target'].head().apply(lambda x: np.array(x).shape))

       start target._np_shape  \
0 2000-01-01        [1024, 3]   
1 2000-01-01        [1024, 3]   
2 2000-01-01        [1024, 3]   
3 2000-01-01        [1024, 3]   
4 2000-01-01        [1024, 3]   

                                              target  
0  [-1.3293273404902177e-07, 1.7617705479648852e-...  
1  [0.8820515850319665, -2.630758929416405, -0.28...  
2  [1.0917233174006584, -0.30767090449730106, 0.1...  
3  [-1.4419568333102197, -0.2799756872675115, -0....  
4  [-0.8402086553986396, -1.5435372346614247, -0....  
0    (3072,)
1    (3072,)
2    (3072,)
3    (3072,)
4    (3072,)
Name: target, dtype: object


In [246]:
# Function to plot multivariate time series
def plot_multivariate_time_series(data, num_rows=3, num_cols=5):
    num_series = num_rows * num_cols
    time_points = np.arange(len(data[0]) // 3)  # 1024 time points for reshaped data
    
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, num_rows * 4))
    axs = axs.flatten()  # Flatten to easily iterate over subplots
    
    for i in range(num_series):
        series = np.array(data[i]).reshape(-1, 3)  # Reshape to [1024, 3]
        for j in range(series.shape[1]):
            axs[i].plot(time_points, series[:, j], label=f'Dimension {j+1}')
        axs[i].set_title(f'Time Series {i+1}')
        axs[i].set_xlabel('Time')
        axs[i].set_ylabel('Value')
        axs[i].legend()
    
    plt.tight_layout()
    plt.show()

# Extract the 'target' column as a list and plot the first 15 multivariate time series
plot_multivariate_time_series(df_ch_mult['target'].head(15).tolist(), num_rows=3, num_cols=5)

## DYNOTEARS Causal Structure

In [ ]:
from causalnex.structure.notears import from_pandas
df_str = df.drop(columns=['Datetime (UTC)'])
sm = from_pandas(df_str)

In [ ]:
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("supporting_files_dynotears/01_fully_connected.html")

In [ ]:
sm.remove_edges_below_threshold(0.8)
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("supporting_files_dynotears/01_thresholded.html")

In [ ]:
from causalnex.structure.notears import from_pandas
df_str = df.drop(columns=['Datetime (UTC)'])
sm = from_pandas(df_str)

In [ ]:
sm.remove_edges_below_threshold(0.8)
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("supporting_files_dynotears/01_thresholded.html")

## Granger causality test with nonlinear forecasting methods

In [ ]:
!pip install nonlincausality

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb  7 23:29:32 2022

@author: Maciej Rosoł

contact: mrosol5@gmail.com, maciej.rosol.dokt@pw.edu.pl
"""
#%%
import os

# os.chdir(os.path.dirname(__file__))
import numpy as np
##import tensorflow
import nonlincausality as nlc
import matplotlib.pyplot as plt
import copy
from nonlincausality.utils import prepare_data_for_prediction, calculate_pred_and_errors
from sklearn.svm import SVR

In [ ]:
#%% Data generation Y->X
np.random.seed(10)
y = (
    np.cos(np.linspace(0, 20, 10_100))
    + np.sin(np.linspace(0, 3, 10_100))
    - 0.2 * np.random.random(10_100)
)
np.random.seed(20)
x = 2 * y ** 3 - 5 * y ** 2 + 0.3 * y + 2 - 0.05 * np.random.random(10_100)
data = np.vstack([x[:-100], y[100:]]).T

plt.figure()
plt.plot(data[:, 0], label="X")
plt.plot(data[:, 1], label="Y")
plt.xlabel("Number of sample")
plt.ylabel("Signals [AU]")
plt.legend()

#%% Test in case of presence of the causality
lags = [50, 150]
data_train = data[:6000, :]
data_val = data[6000:8000, :]
data_test = data[8000:, :]


In [ ]:
results = nlc.nonlincausalityNN(
    x=data_train,
    maxlag=lags,
    NN_config=['d','dr','d','dr'],
    NN_neurons=[100,0.05,100,0.05],
    x_test=data_test,
    run=3,
    epochs_num=[50, 50],
    learning_rate=[0.0001, 0.00001],
    batch_size_num=32,
    x_val=data_val,
    reg_alpha=None,
    callbacks=None,
    verbose=True,
    plot=True,
)

#%% Example of obtaining the results
for lag in lags:
    best_model_X = results[lag].best_model_X
    best_model_XY = results[lag].best_model_XY

    p_value = results[lag].p_value
    test_statistic = results[lag]._test_statistic

    best_history_X = results[lag].best_history_X
    best_history_XY = results[lag].best_history_XY

    nlc.plot_history_loss(best_history_X, best_history_XY)
    plt.title("Lag = %d" % lag)
    best_errors_X = results[lag].best_errors_X
    best_errors_XY = results[lag].best_errors_XY

    cohens_d = np.abs(
        (np.mean(np.abs(best_errors_X)) - np.mean(np.abs(best_errors_XY)))
        / np.std([best_errors_X, best_errors_XY])
    )
    print("For lag = %d Cohen's d = %0.3f" % (lag, cohens_d))
    print(f"Test statistic = {test_statistic} p-value = {p_value}")

    # Using models for prediction
    data_X, data_XY = prepare_data_for_prediction(data_test, lag)
    X_pred_X = best_model_X.predict(data_X)
    X_pred_XY = best_model_XY.predict(data_XY)

    # Plot of true X vs X predicted
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].plot(data_test[lag:, 0], X_pred_X, "o")
    ax[0].set_xlabel("X test values")
    ax[0].set_ylabel("Predicted X values")
    ax[0].set_title("Model based on X")
    ax[1].plot(data_test[lag:, 0], X_pred_XY, "o")
    ax[1].set_xlabel("X test values")
    ax[1].set_ylabel("Predicted X values")
    ax[1].set_title("Model based on X and Y")
    plt.suptitle("Lag = %d" % lag)

    # Another way of obtaining predicted values (and errors)
    X_pred_X, X_pred_XY, error_X, error_XY = calculate_pred_and_errors(
        data_test[lag:, 0], 
        data_X, 
        data_XY, 
        best_model_X, 
        best_model_XY
    )
    # Plot of X predicted vs prediction error
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].plot(X_pred_X, error_X, "o")
    ax[0].set_xlabel("Predicted X values")
    ax[0].set_ylabel("Prediction errors")
    ax[0].set_title("Model based on X")
    ax[1].plot(X_pred_XY, error_XY, "o")
    ax[1].set_xlabel("Predicted X values")
    ax[1].set_ylabel("Prediction errors")
    ax[1].set_title("Model based on X and Y")
    plt.suptitle("Lag = %d" % lag)

In [ ]:
#%% Test in case of absence of the causality
np.random.seed(30)
data_noise = np.vstack([x[:-100], np.random.random(10_000)]).T

lags = [50, 150]
data_noise_train = data_noise[:6000, :]
data_noise_val = data_noise[6000:8000, :]
data_noise_test = data_noise[8000:, :]

results = nlc.nonlincausalityNN(
    x=data_noise_train,
    maxlag=lags,
    NN_config=['d','dr','d','dr'],
    NN_neurons=[100,0.05,100,0.05],
    x_test=data_noise_test,
    run=3,
    epochs_num=[50, 50],
    learning_rate=[0.001, 0.0001],
    batch_size_num=32,
    x_val=data_noise_val,
    reg_alpha=None,
    callbacks=None,
    verbose=True,
    plot=True,
)

#%% Example of obtaining the results
for lag in lags:
    best_model_X_lag50 = results[lag].best_model_X
    best_model_XY_lag50 = results[lag].best_model_XY

    p_value = results[lag].p_value
    test_statistic = results[lag].test_statistic

    best_history_X = results[lag].best_history_X
    best_history_XY = results[lag].best_history_XY

    nlc.plot_history_loss(best_history_X, best_history_XY)
    plt.title("Lag = %d" % lag)

    best_errors_X = results[lag].best_errors_X
    best_errors_XY = results[lag].best_errors_XY

    cohens_d = np.abs(
        (np.mean(np.abs(best_errors_X)) - np.mean(np.abs(best_errors_XY)))
        / np.std([best_errors_X, best_errors_XY])
    )
    print("For lag = %d Cohen's d = %0.3f" % (lag, cohens_d))
    print(f"test statistic = {test_statistic} p-value = {p_value}")
#%% Example of the measure of the causality change over time

data_test_measure = copy.copy(data_test)
np.random.seed(30)
data_test_measure[:1000, 1] = np.random.random(1000)

plt.figure()
plt.plot(data_test_measure[:, 0], label="X")
plt.plot(data_test_measure[:, 1], label="Y")
plt.xlabel("Number of sample")
plt.ylabel("Signals [AU]")
plt.legend()

results = nlc.nonlincausalitymeasureNN(
    x=data_train,
    maxlag=lags,
    window=100,
    step=1,
    NN_config=['d','dr','d','dr'],
    NN_neurons=[100,0.05,100,0.05],
    x_test=data_test_measure,
    run=3,
    epochs_num=[50,50],
    learning_rate=[0.0001, 0.00001],
    batch_size_num=32,
    x_val=data_val,
    verbose=True,
    plot=True,
)


#%% Example of usage for conditional analysis
np.random.seed(30)
z = np.random.random([10_000, 2])

z_train = z[:6000, :]
z_val = z[6000:8000, :]
z_test = z[8000:, :]

results_conditional = nlc.nonlincausalityNN(
    x=data_train,
    maxlag=lags,
    NN_config=['d','dr','d','dr'],
    NN_neurons=[100,0.05,100,0.05],
    x_test=data_test,
    run=1,
    z=z_train,
    z_test=z_test,
    epochs_num=[50, 50],
    learning_rate=[0.0001, 0.00001],
    batch_size_num=32,
    x_val=data_val,
    z_val=z_val,
    reg_alpha=None,
    callbacks=None,
    verbose=True,
    plot=True,
)
# %% Exaple of the usage the package with Scikit-learn model

parametres = {
    'kernel':['poly', 'rbf'],
    'C':[0.01,0.1,1], 
    'epsilon':[0.01,0.1,1.]
}
results_skl = nlc.nonlincausality_sklearn(    
    x=data_train,
    sklearn_model=SVR,
    maxlag=lags,
    params=parametres,
    x_test=data_test,
    x_val=data_val,
    plot=True)

#%% Example of usage other functions for causality analysis

# ARIMA/ARIMAX models
results_ARIMA = nlc.nonlincausalityARIMA(x=data_train[::10], maxlag=[5,15], x_test=data_test[::10])